# Семинар 1. Числа с плавающей точкой. Векторы и их свойства. Нормы

### Пример сложения чисел в формате плавающей точки

- Рассмотрим те же два числа $18.5$ и $6.75$
- Их представление как чисел с плавающей точкой:
    - Двоичное представление $18.5_{10} = 10010.1_2 = 10010.1_2 \cdot 2^0 = 1.00101_2 \cdot 2^4$ даёт мантиссу $00101$
    - Для приведения числа к виду $1.xxxx$ использовали сдвиг влево, поэтому показатель 2 увеличился. Однако может понадобиться сдвигать точку вправо. Тогда показатель экспоненты может стать отрицательным
    - Чтобы этого избежать и хранить её в беззнаковом виде к получившемуся значению прибавляют 127 (для одинарной точности).
    - Тогда $127 + 4 = 131_{10} = 10000011_2$. Также бит, отвечающий за знак равен 0. 
    - В итоге число будет храниться в виде **0 10000011 00101** + необходимые нули
    - Проверка: $2^4 \cdot 1.\{2^{-3} + 2^{-5} \} = 2^4 \cdot 1.15625 = 18.5$
    - Проделав аналогичные манипуляции получим, что $6.75 = 110.11_2 \cdot 2^0 = 1.1011 \cdot 2^2 $ и $2 + 127 = 129_{10} = 10000001_2$. В итоге: $6.75 = $  **0 10000001 1011**
    
- Выполним сложение
    - Для этого надо сделать экспоненты одинаковыми и равными минимальному показателю. В нашем случае минимальный показатель равен 2 и надо сдвинуть на 2 показатель для числа 18.5
    - Как это сделать? Сдвигать биты вправо для увеличения и влево для уменьшения. Проверим: **0 10000001 10100** $= 2^x \cdot 4.\{0.5+0.125\}$, где $x = 1 + 128 - 127 = 2$ 
    - Тогда $6.75 + 18.5 = 0 \ 10000001 \ 1011 + 0 \ 10000001 \ 1010$ но помним о сдвиге на 2 бита!
    - Складываем мантиссы: $1.1011 + 100.1010 = 110.0101 = 110.0101 \cdot 2^2$ - сдвигаем экспоненту на 2
    - Значит в результате имеем $1.100101 \cdot 2^4$ - может пропасть точность, если битов для хранения не хватит! 
    - Или в бинарном виде $0 \ 10000011 \ 100101_2 = 25.25_{10}$
    

## А как будет выглядеть умножение?

Выше показали, как складывать числа в формате плавающей точки. Давайте обсудим нюансы того, как выполняется умножение чисел в таком формате. Для удобства рассмотрим те же самые числа.

- Поскольку основание экспоненты одинаково, то при умножение нужно сложить показатели как двоичные числа
- И перемножить мантиссы тоже как двоичные числа

Пример перемножения целых чисел

$$ \begin{align*} &9 \cdot 5 = 1001_2 \cdot 101_2 = \\ 
& 001001_2 + 100100_2 = 101101_2 = 1 + 4 + 8 + 32 = 45_{10}
\end{align*} 
$$

## Суммирование ряда

- Покажем, почему сложение чисел в формате плавающей точки будет давать разную точность при разном порядке. 
- Рассмотрим следующий замечательный факт

$$
\sum_{n=1}^{\infty} \frac{1}{n^2} = \frac{\pi^2}{6}
$$

и проверим, что он действительно выполяняется с некоторой точность. 

На этом примере проанализируем как лучше складывать такие ряды.

In [4]:
import numpy as np
from numba import jit

N = int(1e6)
items = np.array([1./n**2 for n in range(1, N+1)], dtype=np.float32)
target = np.pi**2 / 6
print(np.sum(items, dtype=np.float32) - target)

@jit(nopython=True)
def forward_sum(series):
    series_val = np.float32(0)
    for it in series:
        series_val += it
    return series_val

def forward_sum_pure(series):
    series_val = np.float32(0)
    for it in series:
        series_val += it
    return series_val

series_val = forward_sum(items)
print(series_val - target)

@jit(nopython=True)
def backward_sum(series):
    series_val = np.float32(0)
    for it in series[::-1]:
        series_val += it
    return series_val

series_val = backward_sum(items)
print(series_val - target)

%timeit forward_sum_pure(items)
%timeit forward_sum(items)


-2.392844625331847e-06
-0.0002087441248377342
-1.0815424402732532e-06
258 ms ± 75.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.38 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Вложения слов в векторное пространство

- Посмотрим на векторные представления слов и то, как векторы, которыми "заменяют" слова сохраняют семантические свойства слов. 
- Это означает, что арифметичесике операции над векторами дают векторы похожие на представления соответствующих слов.

In [5]:
# !pip install gensim
import gensim.downloader as api
# Warning! 2 Gb model will be downloaded!
wv = api.load('word2vec-google-news-300')

In [6]:
for index, word in enumerate(wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [16]:
pairs = [
    ('plane', 'boeing'),
    ('plane', 'airbus'),
    ('plane', 'shuttle'),
    ('plane', 'bombardier'),
    ('plane', 'dog'),
]
for w1, w2 in pairs:
    print('%r \t %r \t %.2f' % (w1, w2, wv.similarity(w1, w2)))

'plane' 	 'boeing' 	 0.40
'plane' 	 'airbus' 	 0.64
'plane' 	 'shuttle' 	 0.32
'plane' 	 'bombardier' 	 0.38
'plane' 	 'dog' 	 0.20


In [19]:
print(wv.most_similar(positive=['plane'], topn=10))

[('airplane', 0.8348400592803955), ('jet', 0.7661097049713135), ('planes', 0.7439613938331604), ('aircraft', 0.7360519170761108), ('jetliner', 0.7206446528434753), ('airliner', 0.7167278528213501), ('Cessna', 0.6842411160469055), ('Plane', 0.6765829920768738), ('propeller_plane', 0.6752769351005554), ('flight', 0.6747332215309143)]


In [28]:
unknown_vec = wv["bird"] - wv["feathers"] + wv["nails"]
sim = wv.cosine_similarities(unknown_vec, wv.vectors)
idx = np.argsort(sim)[::-1]
for i in idx[:10]:
    print(wv.index_to_key[i], sim[i])

nails 0.5771986
bird 0.48011056
dog 0.39971536
rattle_snake 0.3872848
nail 0.385976
puff_adder 0.38555157
rattler 0.37968072
puppy 0.3730222
rattlesnake 0.372675
cat 0.37201318
